# Random predictor for similarity-based methods

### General algorithm

In [1]:
import networkx as nx 
import random
import matplotlib.pyplot as plt

In [2]:
# We want this function to predict k links randomly

def random_prediction_directed(graph, k, multi = False):
    
    n = graph.number_of_nodes()
    nodes = list(graph.nodes)
    links = []
    random.shuffle(nodes) # so that the prediction is not defined by the order of the nodes
    
    while len(links) < k:
        i = random.randint(0,n-1)
        j = random.randint(0,n-1)
        
        if (multi==False and nodes[j] not in [n for n in graph.neighbors(nodes[i])]) or (multi==True): 
            # We don't want to predict edges already existing, if not multigraph
            # if multigraph, we can predict another edge, where we already have one
            if [nodes[i],nodes[j]] not in links : # We don't want to predict the same edge twice
                links.append([nodes[i],nodes[j]])
    
    return links

In [3]:
# We want this function to predict k links randomly

def random_prediction_undirected(graph, k, multi = False):
    
    n = graph.number_of_nodes()
    nodes = list(graph.nodes)
    links = []
    random.shuffle(nodes) # so that the prediction is not defined by the order of the nodes
    
    while len(links) < k:
        i = random.randint(0,n-1)
        j = random.randint(0,n-1)
        
        if (multi==False and nodes[j] not in [n for n in graph.neighbors(nodes[i])]) or (multi==True): 
            # We don't want to predict edges already existing, if not multigraph
            # if multigraph, we can predict another edge, where we already have one
            if [min(nodes[i],nodes[j]),max(nodes[i],nodes[j])] not in links: 
                # We don't want to predict the same edge twice
                links.append([min(nodes[i],nodes[j]),max(nodes[i],nodes[j])])
    
    return links

In [4]:
# function giving MAP for random predictor

def random_results(predictions, actual_edges, graph):
    
    n = graph.number_of_nodes()
    nodes = list(graph.nodes)
    n_edges = graph.number_of_edges()
    test_size = len(actual_edges)
    
    precisions = []
    MAP = 0
    tp = 0
    fp = 0
    
    for k in range(len(predictions)):
            
        if predictions[k] in actual_edges:
            tp += 1
            MAP += (tp /(tp + fp))   
        else:
            fp += 1
                
    if tp == 0:  # Not even one good prediction
        return 0
    
    return(MAP/tp)

In [5]:
# 5-fold cross validation results for random predictor

def CV_random_results(list_graphs, list_targets, network_type='undirected', multi = False):
    test_size = len(list_targets[0])
    MAP = 0
    
    for i in range(100):
        for part in range(5):
        
            if network_type == 'directed':
                predictions = random_prediction_directed(list_graphs[part],test_size,multi)
            else: predictions = random_prediction_undirected(list_graphs[part],test_size,multi)
            
        MAP_N = random_results(predictions,list_targets[part],list_graphs[part])
        MAP = MAP+MAP_N
  
    return(MAP/500)

### First dataset : PROTEINS

The first dataset we work on is a network of the interactions between human proteins.
It makes a perfect example of one of the important applications of link prediction: the
biologic domain. It is a directed, unweighted network composed of 1706 vertices
(proteins) and 6,207 edges (interactions).

In [6]:
# Importation of the dataset: PROTEINS
# http://konect.uni-koblenz.de/networks/maayan-Stelzl 

PROT = []
with open('out.maayan-Stelzl') as inputfile:
    for line in inputfile:
        PROT.append(line.strip().split(','))
PROT = PROT[1:] # list of all the edges
random.shuffle(PROT) # we randomly shuffle the edges

# test size
num_edges_PROT = len(PROT)
num_vertices_PROT = 1706
test_size_PROT = int(num_edges_PROT/5)

# Contains the 5 parts forming the whole dataset
parts_PROT = []

start = 0
end = test_size_PROT
for part in range(5):  # We create the 5 parts
    if end>num_edges_PROT:
        parts_PROT.append(PROT[start:])
    else:parts_PROT.append(PROT[start:end])
    start = end
    end = start + test_size_PROT


In [7]:
PROT_G = nx.DiGraph()
for edge in range(len(PROT)):
    nodes = PROT[edge][0].split(' ')
    PROT_G.add_edge(int(nodes[0]), int(nodes[1])) 
PROT_nodes = PROT_G.nodes()

In [8]:
# graphs contains the 5 different training networks 
# targets contains the target links corresponding

PROT_graphs = []
PROT_targets = []

for i in range(5):     # i is the index of the folder we won't use
    G = nx.DiGraph()
    G.add_nodes_from(PROT_nodes)
    target_links = [] 
    for j in range(5): # We use every other folder
        data = parts_PROT[j]
        if j!=i:        
            for edge in range(len(data)):
                nodes = data[edge][0].split(' ')
                G.add_edge(int(nodes[0]), int(nodes[1])) 
                
                
        else:
            for edge in range(len(data)):
                nodes = data[edge][0].split(' ')
                target_links.append([int(nodes[0]),int(nodes[1])])
                
                

    PROT_graphs.append(G)
    PROT_targets.append(target_links)
  

In [9]:
MAP_R = CV_random_results(PROT_graphs, PROT_targets, 'directed')

In [10]:
MAP_R

0.00035265270711450867

### Second dataset: INFECTIOUS

The second dataset we work on is a network representing the face-to-face behaviour of
people during the exhibition INFECTIOUS: STAY AWAY in 2009 at the Science Gallery
in Dublin. It is undirected, unweighted but has multiple edges. It is composed of 410
vertices (visitors) and 17,298 edges (human face-to-face).

In [11]:
# Importation of the dataset: INFECTIOUS
# http://konect.uni-koblenz.de/networks/sociopatterns-infectious 

INF = []
with open('sociopatterns-infectious\out.sociopatterns-infectious') as inputfile:
    for line in inputfile:
        INF.append(line.strip().split(','))
INF = INF[2:] # list of all the edges
random.shuffle(INF) # we randomly shuffle the edges

# test size
num_edges_INF = len(INF)
num_vertices_INF = 410
test_size_INF = int(num_edges_INF/5)

# Contains the 5 parts forming the whole dataset
parts_INF = []

start = 0
end = test_size_INF
for part in range(5):  # We create the 5 parts
    if end>num_edges_INF:
        parts_INF.append(INF[start:])
    else:parts_INF.append(INF[start:end])
    start = end
    end = start + test_size_INF



In [12]:
INF_G = nx.MultiGraph()
for edge in range(len(INF)):
    nodes = INF[edge][0].split(' ')
    INF_G.add_edge(int(nodes[0]), int(nodes[1])) 
INF_nodes = INF_G.nodes()

In [13]:
# graphs contains the 5 different training networks 
# targets contains the target links corresponding

INF_graphs = []
INF_targets = []

for i in range(5):     # i is the index of the folder we won't use
    G = nx.MultiGraph()
    G.add_nodes_from(INF_nodes)
    target_links = [] 
    for j in range(5): # We use every other folder
        data = parts_INF[j]
        if j!=i:        
            for edge in range(len(data)):
                nodes = data[edge][0].split(' ')
                G.add_edge(int(nodes[0]), int(nodes[1])) 
                
                
        else:
            for edge in range(len(data)):
                nodes = data[edge][0].split(' ')
                # For undirected network, we write the min node first
                # We don't want duplicates because we cannot predict the number of edges to add
                if [min(int(nodes[0]),int(nodes[1])),max(int(nodes[0]),int(nodes[1]))] not in target_links:
                    target_links.append([min(int(nodes[0]),int(nodes[1])),max(int(nodes[0]),int(nodes[1]))])
                
                
    INF_graphs.append(G)
    INF_targets.append(target_links)

In [14]:
MAP_INF_R = CV_random_results(INF_graphs, INF_targets, multi=True)

In [15]:
MAP_INF_R

0.00422435509961273

### Third dataset: ADOLESCENT

The third dataset is also social related. It has been created from a survey that took place
in 1994/1995. Each student was asked to list his 5 best female and his 5 male friends. The
network is composed of 2,539 vertices (students), and 12,969 edges (friendships). It is
directed and weighted. The bigger the weight of an edge is the more important the
relation is.

In [16]:
# Importation of the dataset: ADOLESCENT
# http://konect.uni-koblenz.de/networks/moreno_health

ADO = []
with open('moreno_health\out.moreno_health_health') as inputfile:
    for line in inputfile:
        ADO.append(line.strip().split(','))
ADO = ADO[2:] # list of all the edges
random.shuffle(ADO) # we randomly shuffle the edges

# test size
num_edges_ADO = len(ADO)
num_vertices_ADO = 2539
test_size_ADO = int(num_edges_ADO/5)

# Contains the 5 parts forming the whole dataset
parts_ADO = []

start = 0
end = test_size_ADO
for part in range(5):  # We create the 5 parts
    if end>num_edges_ADO:
        parts_ADO.append(ADO[start:])
    else:parts_ADO.append(ADO[start:end])
    start = end
    end = start + test_size_ADO


In [17]:
ADO_G = nx.DiGraph()
for edge in range(len(ADO)):
    nodes = ADO[edge][0].split(' ')
    ADO_G.add_edge(int(nodes[0]), int(nodes[1]), weight=float(nodes[2]))
ADO_nodes = ADO_G.nodes()

In [18]:
# graphs contains the 5 different training networks 
# targets contains the target links corresponding

ADO_graphs = []
ADO_targets = []

for i in range(5):     # i is the index of the folder we won't use
    G = nx.DiGraph()
    G.add_nodes_from(ADO_nodes)
    target_links = [] 
    for j in range(5): # We use every other folder
        data = parts_ADO[j]
        if j!=i:        
            for edge in range(len(data)):
                nodes = data[edge][0].split(' ')
                G.add_edge(int(nodes[0]), int(nodes[1]), weight=float(nodes[2])) 
                
                
        else:
            for edge in range(len(data)):
                nodes = data[edge][0].split(' ')
                target_links.append([int(nodes[0]),int(nodes[1])])
                
                

    ADO_graphs.append(G)
    ADO_targets.append(target_links)
  

In [19]:
MAP_ADO_R = CV_random_results(ADO_graphs, ADO_targets, 'directed')

In [20]:
MAP_ADO_R

0.0005662475738631031

### Fourth dataset: MISERABLES

The last dataset is entirely different since it is pure fiction. It contains co-occurrences of
characters in Victor Hugo's novel Les Misérables. The network is undirected and positive
weighted. It is composed of 77 vertices (the characters), 254 edges (co-occurrences in a
chapter).

In [21]:
# Importation of the dataset: MISERABLES
# http://konect.uni-koblenz.de/networks/moreno_lesmis

MIS = []
with open('moreno_lesmis\out.moreno_lesmis_lesmis') as inputfile:
    for line in inputfile:
        MIS.append(line.strip().split(','))
MIS = MIS[2:] # list of all the edges
random.shuffle(MIS) # we randomly shuffle the edges

# test size
num_edges_MIS = len(MIS)
num_vertices_MIS = 77
test_size_MIS = int(num_edges_MIS/5)

# Contains the 5 parts forming the whole dataset
parts_MIS = []

start = 0
end = test_size_MIS
for part in range(5):  # We create the 5 parts
    if end>num_edges_MIS:
        parts_MIS.append(MIS[start:])
    else:parts_MIS.append(MIS[start:end])
    start = end
    end = start + test_size_MIS



In [22]:
MIS_G = nx.Graph()
for edge in range(len(MIS)):
    nodes = MIS[edge][0].split(' ')
    MIS_G.add_edge(int(nodes[0]), int(nodes[1]), weight=float(nodes[2]))
MIS_nodes = MIS_G.nodes()

In [23]:
# graphs contains the 5 different training networks 
# targets contains the target links corresponding

MIS_graphs = []
MIS_targets = []

for i in range(5):     # i is the index of the folder we won't use
    G = nx.Graph()
    G.add_nodes_from(MIS_nodes)
    target_links = [] 
    for j in range(5): # We use every other folder
        data = parts_MIS[j]
        if j!=i:        
            for edge in range(len(data)):
                nodes = data[edge][0].split(' ')
                G.add_edge(int(nodes[0]), int(nodes[1]), weight=float(nodes[2])) 
                
                
        else:
            for edge in range(len(data)):
                nodes = data[edge][0].split(' ')
                # For undirected network, we write the min node first
                target_links.append([min(int(nodes[0]),int(nodes[1])),max(int(nodes[0]),int(nodes[1]))])
                
                

    MIS_graphs.append(G)
    MIS_targets.append(target_links)

In [24]:
MAP_MIS_R = CV_random_results(MIS_graphs, MIS_targets)

In [25]:
MAP_MIS_R

0.013220628982735594